# InSAR Time Series Analysis with ISCE2

This notebook performs InSAR time series analysis using the `isce2` stack processor (`topsStack`).

## 1. Setup and Configuration

Define paths to data and working directories.

In [ ]:
import os
import glob
from pathlib import Path

# Base directory
WORK_DIR = "/home/ubuntu/work/slc_stack_test_subset"
os.chdir(WORK_DIR)
print(f"Current working directory: {os.getcwd()}")

# Data paths
SLC_DIR = os.path.join(WORK_DIR, "data/slcs")
ORBIT_DIR = os.path.join(WORK_DIR, "data/orbits")
AUX_DIR = os.path.join(WORK_DIR, "data/aux")
DEM_FILE = os.path.join(WORK_DIR, "dem/demLat_N22_N24_Lon_E094_E097.dem.wgs84")

# Verify paths
for path in [SLC_DIR, ORBIT_DIR, AUX_DIR, DEM_FILE]:
    if not os.path.exists(path):
        print(f"WARNING: Path not found: {path}")
    else:
        print(f"Found: {path}")

## 2. Configure Stack with stackSentinel.py

We use `stackSentinel.py` to set up the processing stack. This will generate a series of run files (shell scripts) to execute the processing steps.

**Parameters:**
- `-s`: Path to SLC directory
- `-d`: Path to DEM file
- `-o`: Path to Orbit directory
- `-a`: Path to Aux directory
- `-b`: Maximum perpendicular baseline (200 m)
- `-t`: Maximum temporal baseline (300 days)
- `-W`: Interferogram workflow (slc/intf)

In [ ]:
# Construct the stackSentinel.py command
cmd = f"stackSentinel.py -s {SLC_DIR} -d {DEM_FILE} -o {ORBIT_DIR} -a {AUX_DIR} -b 200 -t 300 -W interferogram"

print("Running command:")
print(cmd)

# Execute the command
!{cmd}

## 3. Execute Run Files

`stackSentinel.py` generates a `run_files` directory containing scripts like `run_1_unpack`, `run_2_...`, etc. We need to execute these in order.

In [ ]:
run_files_dir = os.path.join(WORK_DIR, "run_files")
if os.path.exists(run_files_dir):
    run_scripts = sorted(glob.glob(os.path.join(run_files_dir, "run_*")))
    print(f"Found {len(run_scripts)} run scripts:")
    for script in run_scripts:
        print(os.path.basename(script))
else:
    print("run_files directory not found. Did stackSentinel.py run successfully?")

In [ ]:
# Function to execute a run script
import subprocess

def execute_run_script(script_path):
    print(f"Executing {script_path}...")
    # Make sure the script is executable
    os.chmod(script_path, 0o755)
    
    # Execute using subprocess
    # We use shell=True to handle environment variables if needed, but direct execution is safer if env is set
    try:
        # Using bash to run the script
        result = subprocess.run(["bash", script_path], check=True, capture_output=True, text=True)
        print(f"Successfully executed {os.path.basename(script_path)}")
        # print(result.stdout) # Uncomment to see full output
    except subprocess.CalledProcessError as e:
        print(f"Error executing {os.path.basename(script_path)}")
        print(e.stderr)
        raise e

# Execute each script in order
# Note: This might take a long time! You might want to run them one by one or in parallel outside the notebook.
for script in run_scripts:
    execute_run_script(script)